## Homework

In [1]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX' -O data.csv

--2022-05-11 12:36:44--  https://docs.google.com/uc?export=download&id=1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX
Resolving docs.google.com (docs.google.com)... 173.194.218.138, 173.194.218.113, 173.194.218.100, ...
Connecting to docs.google.com (docs.google.com)|173.194.218.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/o98p7m30glhbk9nnt5qovtcp2hq9r4j3/1652272575000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download [following]
--2022-05-11 12:36:45--  https://doc-04-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/o98p7m30glhbk9nnt5qovtcp2hq9r4j3/1652272575000/03856158561714992485/*/1OKFSv2GpuUFDphO0r8LdM7bl6MAWwBfX?e=download
Resolving doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs.googleusercontent.com)... 173.194.216.132, 2607:f8b0:400c:c12::84
Connecting to doc-04-ak-docs.googleusercontent.com (doc-04-ak-docs

В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("data.csv", parse_dates=["timestamp"])
df= df.dropna(axis=0)
df.columns

Index(['id', 'timestamp', 'full_sq', 'life_sq', 'floor', 'max_floor',
       'material', 'build_year', 'num_room', 'kitch_sq',
       ...
       'cafe_count_5000_price_2500', 'cafe_count_5000_price_4000',
       'cafe_count_5000_price_high', 'big_church_count_5000',
       'church_count_5000', 'mosque_count_5000', 'leisure_count_5000',
       'sport_count_5000', 'market_count_5000', 'price'],
      dtype='object', length=292)

Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

In [4]:
from sklearn.model_selection import train_test_split
y = np.array(df['price'])


Возможно в ваших моделях вам придется указывать, какие колонки являются категориальными (например, в бустингах). Для упрощения предлагается разделить колонки по следующему принципу:


In [5]:
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]
num_columns = list(set(df.columns).difference(set(cat_columns + drop_columns)))

In [6]:
X = df.drop(columns=['price', 'id', 'timestamp'])
X = pd.get_dummies(X , columns = cat_columns)
X = np.array(X)
X_train, X_test, y_train , y_test = train_test_split(X, y, test_size = 0.20)

### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

In [7]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state=0).fit(X_train, y_train)


Проверьте качество на отложенной выборке.

In [8]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_test, regressor.predict(X_test)))
print(regressor.score(X_test, y_test))

33689689791162.445
0.16211296124231722


### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [9]:
month_year = (df.timestamp.dt.month + df.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df["week_year_cnt"] = week_year.map(week_year_cnt_map)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """


Добавьте следюущие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitchen_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

In [10]:
df['month'] = pd.DatetimeIndex(df['timestamp']).month
df['weekday'] = df['timestamp'].dt.dayofweek
df['floor_divided_by_max_floor'] = (df['floor'].astype(float)) / (df['max_floor'].astype(float)) 
df['kitchenArea_divided_by_fullArea'] = (df['kitch_sq'].astype(float)) / (df['full_sq'].astype(float)) 
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(df.mean(), inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


Разделите выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [47]:
y = np.array(df['price'])
X = df.drop(columns=['price', 'id', 'timestamp'])
X = pd.get_dummies(X , columns = cat_columns)
X = np.array(X)
X_train, X_test, y_train , y_test = train_test_split(X, y, test_size = 0.20)

In [ ]:
np.any(np.isnan(X_train))

False

### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

In [21]:
!pip install catboost

     |████████████████████████████████| 76.6 MB 1.1 MB/s 


In [22]:
from sklearn.ensemble import RandomForestRegressor
import catboost as cb
regressor = DecisionTreeRegressor(random_state=0).fit(X_train, y_train)
forest_regressor = RandomForestRegressor(random_state=0).fit(X_train,y_train)
yandex_regressor = cb.CatBoostRegressor().fit(X_train, y_train)


Learning rate set to 0.049123
0:	learn: 5723891.7030867	total: 107ms	remaining: 1m 46s
1:	learn: 5606829.3453319	total: 156ms	remaining: 1m 17s
2:	learn: 5495464.5388956	total: 206ms	remaining: 1m 8s
3:	learn: 5386235.9496190	total: 252ms	remaining: 1m 2s
4:	learn: 5278663.5422580	total: 298ms	remaining: 59.3s
5:	learn: 5172236.6959470	total: 343ms	remaining: 56.8s
6:	learn: 5077452.2232220	total: 388ms	remaining: 55s
7:	learn: 4985142.7538292	total: 441ms	remaining: 54.7s
8:	learn: 4901161.6326928	total: 490ms	remaining: 53.9s
9:	learn: 4818775.1103268	total: 536ms	remaining: 53s
10:	learn: 4735315.5321394	total: 581ms	remaining: 52.3s
11:	learn: 4660516.0269142	total: 628ms	remaining: 51.7s
12:	learn: 4587389.1471848	total: 681ms	remaining: 51.7s
13:	learn: 4518849.3278541	total: 731ms	remaining: 51.5s
14:	learn: 4456291.4787549	total: 775ms	remaining: 50.9s
15:	learn: 4393474.3477446	total: 821ms	remaining: 50.5s
16:	learn: 4341701.0602606	total: 867ms	remaining: 50.1s
17:	learn: 42

In [23]:
print(mean_squared_error(y_pred=regressor.predict(X_test), y_true=y_test))
print(mean_squared_error(y_pred=forest_regressor.predict(X_test), y_true=y_test))
print(mean_squared_error(y_pred=yandex_regressor.predict(X_test), y_true=y_test))
print(regressor.score(X_test, y_test))
print(forest_regressor.score(X_test, y_test))
print(yandex_regressor.score(X_test, y_test))

29613828650428.668
15275561078375.379
13231055873422.848
0.3546229870652148
0.6670982298156147
0.711654328173494


In [24]:
model_params = {
    'regressor': {
        'model':DecisionTreeRegressor(random_state=0),
        'params' : {
            'max_depth': [5,10,15,20],
            'min_samples_split' : list(range(2,10,2)),
            'min_samples_leaf' : list(range(1,10,1)),
        }  
    },
    'random_forest': {
        'model': RandomForestRegressor(random_state=0),
        'params' : {
             'n_estimators': [1,5,10,50,100],
             'max_depth':[None,1 , 5, 10],
             'min_samples_split': list(range(2,10,2)),
             'min_samples_leaf': list(range(1,10,1)),

        }
    }
}

In [25]:
from sklearn.model_selection import RandomizedSearchCV
scores = []

for model_name, mp in model_params.items():
    clf =  RandomizedSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X_train, y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
parameters_data = pd.DataFrame(scores,columns=['model','best_score','best_params'])
parameters_data

,model,best_score,best_params
0,regressor,0.434078,"{'min_samples_split': 4, 'min_samples_leaf': 3..."
1,random_forest,0.616058,"{'n_estimators': 50, 'min_samples_split': 4, '..."


In [140]:
best_tree = DecisionTreeRegressor(random_state=0)
best_tree.set_params(**parameters_data['best_params'][0])
best_forest = RandomForestRegressor(random_state=0)
best_forest.set_params(**parameters_data['best_params'][1])
best_tree.fit(X_train, y_train)
best_forest.fit(X_train, y_train)
print(best_tree.score(X_test,y_test))
print(best_forest.score(X_test, y_test))

0.46806559182124463
0.6081717451574902


In [ ]:
grid = {  'depth':[3,1,2,6,4,5,7,8,9,10],
          'iterations':[250,100,500,1000],
          'learning_rate':[0.03,0.001,0.01,0.1,0.2,0.3], 
          'l2_leaf_reg':[3,1,5,10,100],
          'border_count':[32,5,10,20,50,100,200],
          } 
model = cb.CatBoostRegressor()         
randomized_search_result = model.randomized_search(grid,
                                                   X=X_train,
                                                   y=y_train,
                                                   plot=True)


In [139]:
best_model_params = randomized_search_result['params']
best_model_yandex = cb.CatBoostRegressor(random_state = 0)
best_model_yandex.set_params(**best_model_params)
best_model_yandex.fit(X_train, y_train)

0:	learn: 5770119.0442398	total: 41.6ms	remaining: 20.7s
1:	learn: 5535450.8072137	total: 80.1ms	remaining: 20s
2:	learn: 5307305.8274144	total: 107ms	remaining: 17.7s
3:	learn: 5113601.7129143	total: 133ms	remaining: 16.4s
4:	learn: 4940981.7673422	total: 158ms	remaining: 15.6s
5:	learn: 4785620.6570520	total: 187ms	remaining: 15.4s
6:	learn: 4642324.4500430	total: 213ms	remaining: 15s
7:	learn: 4499254.5525541	total: 239ms	remaining: 14.7s
8:	learn: 4379823.1449492	total: 269ms	remaining: 14.7s
9:	learn: 4282251.2568235	total: 294ms	remaining: 14.4s
10:	learn: 4182201.0643552	total: 319ms	remaining: 14.2s
11:	learn: 4097285.6951798	total: 345ms	remaining: 14s
12:	learn: 4032105.5287963	total: 371ms	remaining: 13.9s
13:	learn: 3966832.8225518	total: 397ms	remaining: 13.8s
14:	learn: 3903083.8724052	total: 428ms	remaining: 13.8s
15:	learn: 3839114.1613214	total: 453ms	remaining: 13.7s
16:	learn: 3780565.1855634	total: 492ms	remaining: 14s
17:	learn: 3727988.6030077	total: 520ms	remaini

In [29]:
print(best_model_yandex.score(X_test, y_test))

0.6795192561469362


### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

In [117]:
df = pd.get_dummies(df , columns = cat_columns)


KeyError: ignored

Оно начало ругаться, потому что я уже скинул все эти категориальные фичи и попробовал запустить get_dummies ещё раз.

In [142]:
data_invest = df[df["product_type_Investment"]== 1]
data_invest.columns.get_loc('product_type_Investment')

274

In [121]:
X_invest = data_invest.drop(columns=['price', 'timestamp'])
print(X_invest.columns.get_loc('product_type_Investment'))
X_invest = np.array(X_invest)
y_invest = np.array(data_invest['price'])

272


In [113]:
data_owner = df[df["product_type_Investment"]==0]

In [119]:
X_owner = data_owner.drop(columns=['price', 'timestamp'])
X_owner
X_owner = np.array(X_owner)
y_owner = np.array(data_owner['price'])

Случайный лес и Catboost оказались лучшими моделями, мы будем обучать именно их.

In [ ]:
model_invest1= RandomForestRegressor(random_state=0)
model_invest1.set_params(**parameters_data['best_params'][1])
model_invest1.fit(X_invest[:,1:],y_invest) #without id
model_invest2= cb.CatBoostRegressor(random_state=0)
model_invest2.set_params(**best_model_params)
model_invest2.fit(X_invest[:,1:], y_invest)

In [ ]:
model_owner1= RandomForestRegressor(random_state=0)
model_owner1.set_params(**parameters_data['best_params'][1])
model_owner1.fit(X_owner[:,1:],y_owner) #without id
model_owner2= cb.CatBoostRegressor(random_state=0)
model_owner2.set_params(**best_model_params)
model_owner2.fit(X_owner[:,1:], y_owner)

In [149]:
pred_list = []
for i in range(X_test.shape[0]):
  if X_test[i, 274] == 1: #Если это Investment -> пусть предсказывает модель обученная на investment
    pred_list.append(model_invest1.predict(X_test[i, :].reshape(1,X_test.shape[1])))
  else:
     pred_list.append(model_owner1.predict(X_test[i, :].reshape(1,X_test.shape[1])))

In [150]:
pred_yandex_list = []
for i in range(X_test.shape[0]):
  if X_test[i, 274] == 1: #Если это Investment -> пусть предсказывает модель обученная на investment
    pred_yandex_list.append(model_invest2.predict(X_test[i, :].reshape(1,X_test.shape[1])))
  else:
     pred_yandex_list.append(model_owner2.predict(X_test[i, :].reshape(1,X_test.shape[1])))

In [151]:
print(mean_squared_error(y_test, pred_list))
print(mean_squared_error(y_test, pred_yandex_list))

21570419601895.824
17898259998336.52


In [152]:
#До этого
print(mean_squared_error(y_test, best_forest.predict(X_test)))
print(mean_squared_error(y_test, best_model_yandex.predict(X_test)))

14233246461338.338
12689793773138.783


### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).

In [ ]:
# Не успел